In [1]:
from PIL import ImageOps, Image, ImageGrab
#from IPython.display import Image, display 
from numpy import array
import numpy as np
import win32api

import time
import threading
import win32ui, win32gui, win32com, pythoncom, win32con
from win32com.client import Dispatch
from win32api import GetSystemMetrics

import winsound # beep

In [2]:
WINDOW_SUBSTRING = ['- the end -', 'Siebel Sales', 'Flask', 'Почта']

def start_crm():
    ie = Dispatch("InternetExplorer.Application")
    ie.Visible = 1
    
    ie.Navigate('https://crmcorp.ca.sbrf.ru/crmcorp/start.swe?SWECmd=Start&SWEHo=crmcorp.ca.sbrf.ru')

def get_window_info():
    # set window info
    window_info = {}
    #start_crm()
    win32gui.EnumWindows(set_window_coordinates, window_info)
    return window_info

# EnumWindows handler
# sets window coordinates
def set_window_coordinates(hwnd, window_info):
    if win32gui.IsWindowVisible(hwnd):
        for string in WINDOW_SUBSTRING:
            if string in win32gui.GetWindowText(hwnd):
                #rect = win32gui.GetWindowRect(hwnd)          
                rect = win32gui.GetWindowRect(win32gui.GetDesktopWindow())
                x = rect[0]
                y = rect[1]
                w = rect[2] - x
                h = rect[3] - y
                window_info['x'] = x
                window_info['y'] = y
                window_info['width'] = w
                window_info['height'] = h
                window_info['name'] = win32gui.GetWindowText(hwnd)
                win32com.client.Dispatch("WScript.Shell").SendKeys('%')
                #start_crm()
                win32gui.SetForegroundWindow(hwnd)
                win32gui.SetWindowPos(hwnd, win32con.HWND_NOTOPMOST, x, y, w, h, 0)

def get_screen(size=win32gui.GetWindowRect(win32gui.GetDesktopWindow())):
    time.sleep(3)
    screen = ImageGrab.grab(size)
    screen.save(r'.\screen\full.png', 'PNG')   
    img = array(screen.getdata(), dtype='int32').reshape((screen.size[1], screen.size[0], 3))
    return img


def load_image(infilename):
    img = Image.open(infilename)
    img.load()
    data = np.asarray(img, dtype="int32")
    return data

def save_image(npdata, outfilename):
    img = Image.fromarray(np.asarray(np.clip(npdata,0,255), dtype="uint32"), "L")
    img.save(outfilename)

    
def click(arg):
    # click at position x, y
    win32api.SetCursorPos((arg[1],arg[0]))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,arg[1],arg[0],0,0) # press
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,arg[1],arg[0],0,0) # up
    time.sleep(3)
    
def finder(matrix, pattern):
    
    def comparer(r, c):
        if c + pattern.shape[1] > matrix.shape[1] or r + pattern.shape[0] > matrix.shape[0]:
            return
        if (np.subtract(matrix[r:r+pattern.shape[0], c:c+pattern.shape[1]], pattern) == 0).all():
            return True
        return
    
    def cmp():
        for row, sub in enumerate(matrix):
            for column, elem in enumerate(sub):
                if (pattern[0][0] == elem).all(): # corner element
                    if comparer(row, column):
                        return row, column
    return cmp()

In [3]:
get_window_info(), winsound.Beep(2000, 200)

({}, None)

In [47]:
get_window_info()

#size = win32gui.GetWindowRect(win32gui.GetDesktopWindow())
try:
    click(finder(get_screen(), load_image(r'.\screen\make.png')))    
except:
    #winsound.Beep(2500, 200)
    pass

try:
    click(finder(get_screen(), load_image(r'.\screen\send.png')))
    get_screen()
    #winsound.Beep(2000, 200)
except:
    pass

try:
    if finder(get_screen(), load_image(r'.\screen\mistake.png')):
        click(finder(get_screen(), load_image(r'.\screen\undu.png')))
    get_screen()
    #winsound.Beep(2000, 200)
except:
    pass

    #winsound.Beep(2500, 200)
winsound.Beep(2000, 200)

In [6]:
def finder(matrix, pattern):
    
    def comparer(r, c):
        if c + pattern.shape[1] > matrix.shape[1] or r + pattern.shape[0] > matrix.shape[0]:
            return
        if (np.subtract(matrix[r:r+pattern.shape[0], c:c+pattern.shape[1]], pattern) == 0).all():
            return True
        return
    
    def cmp():
        for row, sub in enumerate(matrix):
            for column, elem in enumerate(sub):
                if (pattern[0][0] == elem).all(): # corner element
                    if comparer(row, column):
                        return row, column
    return cmp()

In [6]:
#%%time
#(np.subtract(matrix[681:681+pattern.shape[0], 644:644+pattern.shape[1]], pattern)==0).all()

#np.all((np.take(seq, check) == subseq), axis=-1)

#np.where(
#np.dot(pattern, np.moveaxis(pattern, [0, 1, 2], [-1, -2, -3]))
# [-2, -3, -1]
# [-3, -2, -1]
# [-1, -2, -3]
# [-2, -1, -3]
# [-1, -3, -2] 4d
# [-3, -1, -2] 4d
#== target)[0]
#matrix[681, 644] 

In [4]:
#test load and find    
matrix = load_image(r'.\screen\full.png')
pattern = load_image(r'.\screen\send.png')
matrix.shape, pattern.shape

((768, 1366, 3), (13, 70, 3))

In [ ]:
%%time
finder(matrix, pattern) # find and click

In [ ]:
def find_subsequence(seq, subseq):
    target = np.dot(subseq, subseq)
    candidates = np.where(np.correlate(seq,
                                       subseq, mode='valid') == target)[0]
    # some of the candidates entries may be false positives, double check
    check = candidates[:, np.newaxis] + np.arange(len(subseq))
    mask = np.all((np.take(seq, check) == subseq), axis=-1)
    return candidates[mask]

find_subsequence(img.flatten(), find_.flatten())

In [ ]:
if win32con.WM_LBUTTONUP:
    print("You clicked me.")
elif win32con.WM_LBUTTONDBLCLK:
    print("You double-clicked me - goodbye")
    win32gui.DestroyWindow(self.hwnd)
elif win32con.WM_RBUTTONUP:
    print("You right clicked me.")
    menu = win32gui.CreatePopupMenu()